# Prediction based on Machine Learning Models

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import csv

from pricing_data import krx_data

## 1. Data Description
- index = 'date', 'ticker'
- Target: 'target'
- Variables:
  - Market return based: 'market_return', 'excess_market_return'
  - Fama French 3 factor: 'ff3_bin_return', 'smb', 'hml', 
  - CAPM: 'const', 'beta', 'ido_vol', 'beta_seq', 
  - Fundamental: 'EPR', 'BPR', 'div_ret','div'
  - ETC:
    - 'size_rnk', 'share_turnover', 'share_turnover_rnk', 'std12', 'cross_rnk', 'time_rnk',
    - Momentum:'mom1', 'mom2', 'mom3', 'mom4', 'mom5', 'mom6', 'mom7', 'mom8', 'mom9', 'mom10', 'mom11', 'mom12', 
    - Support line(based on price): 'support_low', 'support_high'
  - Macro
    - Korean Treasury: 'tb3y', 'tb5y', 'tb10y', 'cb3y'
    - usd/krw: 'change_usd_krw_monthly', 'lo_usd_krw_monthly', 'ho_usd_krw_monthly', 'co_usd_krw_monthly', 'change_usd_krw_daily', 'lo_usd_krw_daily', 'ho_usd_krw_daily', 'co_usd_krw_daily',
    - WTI: 'change_wti', 'lo_wti', 'ho_wti', 'co_wti', 
    - Market Portfolio: 'change_nasdaq', 'lo_nasdaq', 'ho_nasdaq', 'co_nasdaq', 'close_sp500', 'change_sp500', 'lo_sp500', 'ho_sp500', 'co_sp500',
    - US Treasury: 'close_bond_10y', 'close_bond_2y', 'close_bond_1m', 'close_bond_1y', 
    - VIX:'close_vix', 'change_vix', 'lo_vix', 'ho_vix', 'co_vix',
  - Log:'log_mom1', 'log_mom2', 'log_mom3', 'log_mom4', 'log_EPR', 'log_share_turnover', 'log_mom6', 'log_mom5', 'log_mom12', 'log_mom11', 'log_mom10', 'log_mom8', 'log_mom9', 'log_mom7', 'log_std12', 'log_BPR', 'log_change_wti', 'log_ff3_bin_return', 'log_ho_wti', 'log_ho_usd_krw_monthly', 'log_smb', 'log_co_sp500', 'log_change_usd_krw_daily', 'log_ho_vix', 'log_change_vix', 'log_change_usd_krw_monthly', 'log_ho_usd_krw_daily', 'log_ho_sp500', 'log_close_vix', 'log_ho_nasdaq', 'log_co_usd_krw_daily', 'log_close_bond_1m', 'log_change_sp500', 'log_co_nasdaq', 'log_close_bond_1y', 'log_close_bond_2y', 'log_ido_vol', 'log_change_nasdaq', 'log_close_sp500', 'log_lo_usd_krw_daily', 'log_lo_nasdaq', 'log_lo_sp500', 'log_lo_usd_krw_monthly', 'log_beta_seq', 'log_beta', 'log_hml', 'log_co_usd_krw_monthly', 'log_lo_wti', 
  - Categorical: 'vix_cat_mid', 'vix_cat_high'

In [2]:
train, test, test_years = krx_data()

print(train[test_years[-1]][1].shape)
test_years

Data Split: 100%|██████████| 15/15 [00:00<00:00, 18.38it/s]

(109799, 118)


Int64Index([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
            2018, 2019, 2020, 2021],
           dtype='int64', name='date')

## 3. Machine Learning Models

In [3]:
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.pipeline import Pipeline

# from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from MLutils.SklearnUtils import *

$Out\ of\ Sample\ R^2$ by Campbel and Thomson

In [4]:
def r_os(y_real, y_pred): # 
    return 1 - ((y_real-y_pred)**2).sum() /((y_real- y_real.mean())**2).sum()

    # mse_m = ((y_pred.reshape(-1) - y_real)**2).mean()
    # mse_bm = (y_real**2).mean()
    # return 1 - mse_m / mse_bm

r2os = make_scorer(r_os, greater_is_better=True)

In [5]:
rnd = 77  # random_state


class Regressions:
    def __init__(self, year, x, y, x_test, y_test):
        self.y = y.values
        self.x = x.values
        self.y_test = y_test.values.reshape(-1, 1)
        self.x_test = x_test.values
        self.year = year

        self.models = ['lin', 'en', 'pls', 'pcr', 'rf', 'gbr']
        self.res_cols = ['year', 'name', 'params', 'pred_R2_OOS', 'train_score',
                         'CV_R2', 'CV_MSE', 'complexity']

        self.error = []
        self.res = []
        self.result = pd.DataFrame()
        
    def reset(self, year, x, y, x_test, y_test):
        self.y = y.values
        self.x = x.values
        self.y_test = y_test.values.reshape(-1, 1)
        self.x_test = x_test.values
        self.year = year
        
        self.error = []
        self.res = []

    def execute(self, model, params, name):
        reg, _ = learning(self.x, self.y, regressor=model, params=params, pred=False, scores=[
                          'r2', 'neg_mean_squared_error'], refit='r2', verbose = 0)

        if name == 'en':
            complexity = (reg.best_estimator_.coef_ != 0).sum()
        elif name == 'rf':
            complexity = np.mean(
                [estimator.tree_.max_depth for estimator in reg.best_estimator_.estimators_])
        elif name == 'gbr':
            complexity = np.mean(
                [estimator.max_features_ for estimator in reg.best_estimator_.estimators_.reshape(-1)])
        elif name == 'pls':
            complexity = reg.best_params_
        elif name == 'pcr':
            complexity = reg.best_params_['pca__n_components']

        pred_score = r2os(reg, self.x_test, self.y_test)

        # year, name, best CV result, prediction score (R^2 OOS),
        #   training set score(R^2), CV result(R^2), CV result(MSE), complexity
        self.res.append([self.year, name, reg.best_params_, pred_score,
                        reg.score(self.x, self.y), reg.best_score_, -reg.cv_results_['mean_test_neg_mean_squared_error'], complexity])

        return reg

    @property
    def lin(self):
        model = LinearRegression(fit_intercept=True)
        model.fit(self.x, self.y)
        score = model.score(self.x, self.y)
        mse = mean_squared_error(self.y, model.predict(self.x))
        pred_score = r2os(model, self.x_test, self.y_test)
        self.res.append(
            [self.year, 'lin', 0, pred_score, r2_score(self.y, model.predict(self.x)),
             score, mse, 0])

    @property
    def en(self):
        model = ElasticNet(max_iter=10000, random_state=rnd)
        params = {'alpha': np.linspace(
            20, 150, 10), 'l1_ratio': np.linspace(0.1, 1, 10)}
        reg = self.execute(model, params, 'en')
        return reg

    @property
    def pcr(self):
        model = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA()),
            ('linear', LinearRegression(fit_intercept=True))])
        params = {'pca__n_components': np.arange(1, 10)}  # self.x.shape[1]+1)}
        reg = self.execute(model, params, 'pcr')
        return reg

    @property
    def pls(self):
        model = PLSRegression(random_state=rnd)
        params = {'n_components': np.arange(1, self.x.shape[1]+1)}
        reg = self.execute(model, params, 'pls')
        return reg

    @property
    def rf(self):
        dt = RandomForestRegressor(random_state=rnd).fit(x, y)
        model = RandomForestRegressor(random_state=rnd)
        params = {
            'n_estimators': [10, 20, 50,],
            'max_depth': np.arange(1, dt.tree_.max_depth+1, int(dt.tree_.max_depth/5)),
        }
        reg = self.execute(model, params, 'rf')
        return reg

    @property
    def gbr(self):
        model = GradientBoostingRegressor(random_state=rnd)
        params = {
            'learning_rate': np.linspace(0.01, 0.1, 3),
            'max_depth': [3,7,10,20, 50],
            # 'max_features': range(1, len(dt1.feature_importances_)+1)
        }
        reg = self.execute(model, params, 'gbr')
        return reg

    def fit_all(self):
        try:
            # for i in self.models:
            #     getattr(self, i)
            i = 'lin'
            getattr(self, i)
        except:
            print(f'Error while model {i} in {self.year}')
            self.error.append((self.year, i))

        if len(self.error) != 0:
            with open('error.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerows(self.error)

        result = pd.DataFrame(self.res, columns=self.res_cols)
        if len(self.result) != 0:
            result.to_csv('result.csv', mode='a', header=False, index=False)
        else:
            result.to_csv('result.csv', index=False)

        self.result = pd.concat([self.result, result], axis = 0)



In [6]:
start = True
pbar = tqdm(test_years)
for year in pbar:
    pbar.set_description(f"Processing {year}")
    y, x = train[year]
    y_test, x_test = test[year]
    if start == True:
        mod = Regressions(year, x, y, x_test, y_test)
        start = False
    else:
        mod.reset(year, x, y, x_test, y_test)
    
    mod.fit_all()


Processing 2021: 100%|██████████| 15/15 [00:27<00:00,  1.82s/it]


In [9]:
mod.result

,year,name,params,pred_R2_OOS,train_score,CV_R2,CV_MSE,complexity
0,2007,lin,0,-2.910209e+07,0.252776,0.252776,3.190037,0
0,2008,lin,0,-8.885779e+09,0.239861,0.239861,2.973091,0
0,2009,lin,0,-8.360745e+08,0.241650,0.241650,2.826106,0
0,2010,lin,0,-3.193419e+08,0.428791,0.428791,2.048899,0
0,2011,lin,0,-1.792582e+06,0.017706,0.017706,0.665855,0
0,2012,lin,0,-2.255555e+06,0.019353,0.019353,0.542659,0
0,2013,lin,0,-8.772988e+07,0.026582,0.026582,0.301652,0
0,2014,lin,0,-2.085100e+06,0.026734,0.026734,0.239406,0
0,2015,lin,0,-6.991220e+05,0.020513,0.020513,0.191733,0
0,2016,lin,0,-7.605314e+05,0.045877,0.045877,0.137392,0


In [16]:
 reg = GradientBoostingRegressor(random_state=rnd, max_depth = 50, n_estimators=100).fit(x, y)

KeyboardInterrupt: 

In [15]:
reg.score(x_test,y_test)

-0.303447192240039

In [8]:
np.round(np.geomspace(2, 10, 5))

array([ 2.,  3.,  4.,  7., 10.])